In [45]:
from datasets import load_dataset
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel, BertTokenizer, BertModel, BertForSequenceClassification

Use CLIP model of OpenAI

In [48]:
from datasets import load_dataset

# Tải bộ dữ liệu từ file local với trust_remote_code=True
dataset = load_dataset("../fish_tracking_dataset.py", trust_remote_code=True)

# In thông tin về bộ dữ liệu
print(dataset)

# Truy cập phần train của bộ dữ liệu
train_dataset = dataset['train']

# In một vài mẫu đầu tiên
for i, example in enumerate(train_dataset.select(range(5))):
  print(f"Mẫu {i + 1}:")
  print(f"  ID: {example['id']}")
  print(f"  Bounding Boxes: {example['bboxes']}")
  print(f"  Kích thước ảnh: {example['image'].size}")
  print(f"  Kích thước mask: {example['mask'].size}")
  print()

# Đếm số lượng mẫu trong bộ dữ liệu
print(f"Tổng số mẫu trong bộ dữ liệu: {len(train_dataset)}")

Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 99 examples [00:01, 64.42 examples/s] 


DatasetDict({
    train: Dataset({
        features: ['id', 'image', 'mask', 'bboxes'],
        num_rows: 99
    })
})
Mẫu 1:
  ID: 1
  Bounding Boxes: [{'type': 'box', 'label': 'fish', 'points': [(115.06, 284.8), (249.0, 394.0)]}, {'type': 'box', 'label': 'fish', 'points': [(581.88, 289.74), (683.72, 393.12)]}, {'type': 'box', 'label': 'fish', 'points': [(583.38, 478.65), (769.57, 599.53)]}, {'type': 'box', 'label': 'fish', 'points': [(1014.1, 500.26), (1106.98, 625.76)]}, {'type': 'box', 'label': 'fish', 'points': [(203.38, 610.53), (314.48, 778.21)]}, {'type': 'box', 'label': 'fish', 'points': [(375.68, 766.89), (510.44, 836.84)]}, {'type': 'box', 'label': 'fish', 'points': [(1130.1, 184.81), (1257.22, 270.53)]}, {'type': 'box', 'label': 'fish', 'points': [(1303.47, 332.17), (1362.19, 390.46)]}, {'type': 'box', 'label': 'fish', 'points': [(860.56, 406.65), (965.02, 536.78)]}]
  Kích thước ảnh: (1752, 986)
  Kích thước mask: (1752, 986)

Mẫu 2:
  ID: 2
  Bounding Boxes: [{'type': 'bo

Use model CLIP of OpenAI

In [16]:
# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [40]:
image = Image.open("../data/images/001.jpg")

# Các mô tả tình huống nguy hiểm có thể gặp trong nuôi trồng thủy sản
descriptions = [
  "Ao nuôi có độ pH thấp, và cá đang trong tình trạng căng thẳng.",
  "Ao nuôi có độ pH ở mức lành mạnh, và cá đang bơi bình thường."
]

inputs = processor(text=descriptions, images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
 
# In xác suất tương đồng
for i, description in enumerate(descriptions):
  print(f"Description: {description}, Probability: {probs[0][i].item():.4f}")

Description: Ao nuôi có độ pH thấp, và cá đang trong tình trạng căng thẳng., Probability: 0.6260
Description: Ao nuôi có độ pH ở mức lành mạnh, và cá đang bơi bình thường., Probability: 0.3740


In [26]:
# Select alarm thresholds and status notifications
threshold = 0.6  # Ngưỡng để kích hoạt cảnh báo
alert_triggered = False

for i, description in enumerate(descriptions):
  if probs[0][i].item() > threshold and description != "ao nuôi bình thường":
    print(f"Cảnh báo: {description} phát hiện với xác suất {probs[0][i].item():.4f}")
    alert_triggered = True

if not alert_triggered:
  print("Tình trạng bình thường, không có cảnh báo.")

Cảnh báo: The pond has healthy pH levels, and the fish are swimming normally. phát hiện với xác suất 0.6138


Use model bert

In [13]:
# Load the bert-base-uncased model and tokenizer
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Prepare the input text
text = "Mức oxy trong nước rất thấp, có nhiều cá chết trong ao."
inputs = tokenizer(text, return_tensors="pt")

outputs = model(**inputs)
print(outputs)

# Lấy kết quả phân loại
# logits = outputs.logits
# predicted_class = torch.argmax(logits, dim=1).item()
# print(f"Predicted class: {predicted_class}")

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.7187,  0.1318, -0.0275,  ...,  0.1975,  0.5841,  0.8451],
         [ 0.2540, -0.0051,  0.0949,  ...,  0.2876,  0.3635,  1.3087],
         [ 0.2648,  0.4071,  0.2541,  ...,  0.3341,  0.1399,  0.9385],
         ...,
         [-0.0805,  0.7691,  0.6438,  ...,  0.2144,  0.0900,  0.6337],
         [-0.5273, -0.2463, -0.4941,  ...,  0.7006,  0.0510, -0.0650],
         [ 0.6237, -0.0661, -0.2781,  ..., -0.1473, -0.2327,  0.0360]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.7908, -0.4555, -0.9864,  0.9068,  0.6622, -0.2971,  0.8909,  0.3707,
         -0.9134, -1.0000, -0.7463,  0.9765,  0.9602,  0.7717,  0.7554, -0.8597,
         -0.5980, -0.6678,  0.4695, -0.4431,  0.6579,  1.0000, -0.4635,  0.1826,
          0.4511,  0.9948, -0.7041,  0.7696,  0.8901,  0.5060, -0.7820,  0.3110,
         -0.9770, -0.3276, -0.9917, -0.9833,  0.5587, -0.3580, -0.2350,  0.0061,
         -0.7735,  0.4807,  1.00